In [1]:
# PyTorch の基本モジュールをインポート
import torch
import torch.nn.functional as F

# GATConv: グラフ注意ネットワーク（GAT）のための畳み込みレイヤー
from torch_geometric.nn import GATConv

# Planetoid: Cora, Citeseer, Pubmed などのベンチマーク用グラフデータセットを提供するモジュール
from torch_geometric.datasets import Planetoid

In [2]:
# Planetoid データセット（ここでは 'Cora'）をダウンロードし、指定パスに保存する
# 'root' は保存先ディレクトリ、'name' は使用するデータセット名を指定
dataset: Planetoid = Planetoid(root="/tmp/Cora", name="Cora")

# データセットは1つの大きなグラフで構成されているため、インデックス 0 で取り出す
data = dataset[
    0
]  # data は1つのグラフデータを保持しており、x, edge_index, y などの属性を持つ

In [5]:
import torch.nn as nn
from torch_geometric.data import Data

# グラフ注意ネットワーク（GAT）の定義
class GAT(nn.Module):
    def __init__(
        self, in_d: int, mid_d: int, out_d: int, heads: int, dropout: float = 0.6
    ) -> None:
        """
        GAT モデルの初期化

        Parameters:
        - in_d (int): 入力特徴量の次元数（ノード特徴量の次元）
        - mid_d (int): 中間層の出力次元数（各ヘッドの出力次元）
        - out_d (int): 出力層の次元数（分類クラス数など）
        - heads (int): 注意ヘッドの数（マルチヘッドアテンション）
        - dropout (float): ドロップアウト率
        """
        super().__init__()
        # 第1層: GATConv（マルチヘッドアテンションあり）
        self.conv1 = GATConv(in_d, mid_d, heads=heads, dropout=dropout)
        # 第2層: GATConv（ヘッド統合後の次元数を入力とする）
        self.conv2 = GATConv(mid_d * heads, out_d, dropout=dropout)

    def forward(self, data: Data) -> torch.Tensor:
        """
        フォワードパス

        Parameters:
        - data (Data): PyG の Data オブジェクト（x, edge_index などを含む）

        Returns:
        - torch.Tensor: 出力ロジット（対数ソフトマックス済み）
        """
        # ノードの特徴量とエッジ情報を取得
        x, edge_index = data.x, data.edge_index

        # 第1層 GATConv ＋ 活性化関数（ELU）
        x = self.conv1(x, edge_index)
        x = F.elu(x)

        # 第2層 GATConv（出力）
        x = self.conv2(x, edge_index)

        # クラスごとの対数確率を出力
        return F.log_softmax(x, dim=1)

In [6]:
# GATモデルのインスタンス化
# 入力次元: 各ノードの特徴量の次元（dataset.num_node_features）
# 中間層の出力次元: 各ヘッドにつき8次元
# 出力次元: クラス数（dataset.num_classes）
# ヘッド数: 8（マルチヘッドアテンション）
model = GAT(dataset.num_node_features, 8, dataset.num_classes, 8)

In [7]:
# 最適化手法として確率的勾配降下法（SGD）を設定
# - model.parameters(): 学習対象のGATモデルのパラメータ群
# - lr=0.1: 学習率（各ステップでのパラメータの更新量）
# - weight_decay=1e-4: L2正則化（過学習防止のための項）
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

In [8]:
def train(epoch: int) -> None:
    """
    GATモデルを指定されたエポック数だけ学習する関数。

    Parameters:
    epoch (int): 学習を繰り返すエポック数
    """
    model.train()  # モデルを学習モードに設定（DropoutやBatchNormが有効になる）

    for i in range(epoch):
        optimizer.zero_grad()  # 勾配の初期化（前回のバックプロパゲーションの勾配をリセット）

        out = model(data)  # 入力データに対する予測（各ノードのクラスごとの確率）

        # 負の対数尤度損失（学習用マスクがTrueのノードだけを対象に計算）
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])

        loss.backward()  # 誤差逆伝播による勾配計算
        optimizer.step()  # モデルのパラメータを更新

In [9]:
%time train(500)

CPU times: user 9.85 s, sys: 3.97 s, total: 13.8 s
Wall time: 8.18 s


In [10]:
# モデルを評価モードに設定（DropoutやBatchNormを無効にする）
model.eval()

# 各ノードに対する予測ラベルを取得（クラスごとの確率の最大値のインデックスを取得）
pred: torch.Tensor = model(data).argmax(dim=1)

# テストデータにおける正解数をカウント
correct: torch.Tensor = (pred[data.test_mask] == data.y[data.test_mask]).sum()

# テストデータ全体に対する正解率（accuracy）を計算
acc: float = int(correct) / int(data.test_mask.sum())

# 結果（正解率）を表示
acc

0.807